<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
TIME SERIES DATA PROCESSING
<br>
AVAILABILITY FACTOR | SCALED OUTFLOWS
</div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
Main Formatting Notebook
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to proccess the raw data for the Availability Factor Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs
import datetime
from bs4 import BeautifulSoup
import http.client
from multiprocessing import Pool
import shutil
from datetime import datetime

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Indicate the year of all data is referring to in the variable data_year.
</div>

In [3]:
# Year to which data refers to:
data_year = 2024

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
The four standar_time variables (UTC, WET/WEST, CET/CEST and EET/EST) can be defined in order to use any of the raw data since it is available for each of standard times.
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
To set this just uncomment the corresponding line leaving the others commented.
</div>

In [4]:
# Universal standad time:
standard_time = 'UTC'

# Western European Time:
#standard_time = 'WET_WEST'

# Central European Time:
#standard_time = 'CET_CEST'

# Eastern European Time:
#standard_time = 'EET_EST'

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Defining the countries and its acronyms modeled in Dispa-SET
</div>

In [5]:
# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

dispaSET_codes = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "EL", "ES", "FI", "FR", "HR", "HU", 
                  "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"
]

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
4. Availability Factor and Scaled Outflow and Raw Data Directories Definition
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Defining the path to the folders that are going to content all the data realted to the availability factor and scaled outflow time series.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Futher the raw data is going to be used to get both time series, availability factor and scaled inflow. 
</div>

In [6]:
# Additional string to be appended
additional_path = "/RawData/AvailabilityFactors/"
additional_path_1 = "/RawData/HydroData/ScaledInflows/ScaledOutflows"
additional_path_2 = "Time_Series_Raw_Data_Source/"
#additional_path_3 = os.path.join(additional_path_2, standard_time)

# Construct the standard_time_data_folder_path variable
standard_time_data_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Availability_Factor_folder_path variable
availability_factors_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Scaled_OutFlows_folder_path variable
scaled_outflows_folder_path = dispaSET_unleash_folder_path + additional_path_1

# Construct the Raw Data Source variable
raw_rata_source_folder_path = availability_factors_folder_path + additional_path_2

print("standard_time_data_folder_path:", standard_time_data_folder_path)
print("availability_factors_folder_path:", availability_factors_folder_path)
print("scaled_outflows_folder_path:", scaled_outflows_folder_path)
print("raw_rata_source_folder_path:", raw_rata_source_folder_path)

standard_time_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/
availability_factors_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/
scaled_outflows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/ScaledOutflows
raw_rata_source_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    4.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [7]:
# Define the paths
additional_path_6 = '/RawData/AvailabilityFactors_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_6

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary

shutil.copytree(availability_factors_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    5. Master File
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Defining a csv file where all the needed paths and auxiliary information in the next processing steps are going to be saved.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
The name of the csv file is defined by the variable 'standard_time_data_file_name' and its path is defined by the varibale 'standard_time_data_file_path'
</div>

In [8]:
standard_time_data_file_name = 'Standard_Time_Data_Source.csv'

standard_time_data_file_path = os.path.join(standard_time_data_folder_path, standard_time_data_file_name)

# Create the CSV file if it doesn't exist
if not os.path.exists(standard_time_data_file_path):
    open(standard_time_data_file_path, 'w').close()

print(standard_time_data_file_path)

def extract_folder_path(file_path):
  """Extracts the folder path from a file path.

  Args:
    file_path: The full path to the file.

  Returns:
    The folder path as a string.
  """

  return os.path.dirname(file_path)

print(standard_time_data_folder_path)

/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Standard_Time_Data_Source.csv
/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [9]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"standard_time:                                             {standard_time}")
print (f"standard_time_data_folder_path:                            {standard_time_data_folder_path}")
print (f"availability_factors_folder_path:                          {availability_factors_folder_path}")
print (f"scaled_outflows_folder_path:                               {scaled_outflows_folder_path}")
print (f"standard_time_data_file_path:                              {standard_time_data_file_path}")
print (f"standard_time_data_file_name:                              {standard_time_data_file_name}")
print (f"raw_rata_source_folder_path:                               {raw_rata_source_folder_path}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2024
standard_time:                                             UTC
standard_time_data_folder_path:                            /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/
availability_factors_folder_path:                          /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/
scaled_outflows_folder_path:                               /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/ScaledOutflows
standard_time_data_file_path:                              /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Standard_Time_Data_Source.csv
standard_time_data_file_name:                              Standard_Time_Data_Source.csv
raw_rata_source_folder_path:                               /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFa

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Writting the 'Country', 'Dispa-SET_Code', 'Standard_Time' and 'Data_Year' columns to the 'standard_time_data_file_path'
</div>

In [10]:
def create_csv_file(standard_time_data_file_path, countries, dispaSET_codes, standard_time, data_year):
    """Creates a CSV file with columns for Country, Dispa-SET_Code, Standard_Time, and Data_Year.

    Args:
        standard_time_data_file_path: The path to the CSV file to be created.
        countries: A list of country names.
        dispaSET_codes: A list of Dispa-SET codes.
        standard_time: The standard time value.
        data_year: The year to be included in the new column.
    """

    # Ensure lists have the same length by truncating or padding with None
    min_length = min(len(countries), len(dispaSET_codes))
    countries = countries[:min_length]
    dispaSET_codes = dispaSET_codes[:min_length]

    data = {
        'Country': countries,
        'Dispa-SET_Code': dispaSET_codes,
        'Standard_Time': [standard_time] * min_length,
        'Data_Year': [data_year] * min_length
    }
    df = pd.DataFrame(data)
    df.to_csv(standard_time_data_file_path, index=False)

    # Print a message indicating the file has been created
    print(f"CSV file created at {standard_time_data_file_path} with {min_length} rows.")

# Example usage
# standard_time_data_file_path = '/path/to/standard_time_data.csv'
# countries = ['Country1', 'Country2', 'Country3']
# dispaSET_codes = ['Code1', 'Code2', 'Code3']
# standard_time = '2023-01-01T00:00:00Z'
# data_year = '2023'
create_csv_file(standard_time_data_file_path, countries, dispaSET_codes, standard_time, data_year)

CSV file created at /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Standard_Time_Data_Source.csv with 30 rows.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Writting the 'Time_Series_Raw_Data_File_Path' column to the 'standard_time_data_file_path'.
<br>
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
This columns specify the path to the corresponding raw data csv file sources.
</div>

In [11]:
# Read the CSV file into a DataFrame
df = pd.read_csv(standard_time_data_file_path)

# Create the new column with the required file paths
df['Time_Series_Raw_Data_File_Path'] = df.apply(
    lambda row: f"{raw_rata_source_folder_path}{standard_time}/{row['Dispa-SET_Code']}/{data_year}_1.csv", axis=1)

# Save the updated DataFrame back to a CSV file
df.to_csv(standard_time_data_file_path, index=False)

print(f"Updated CSV file saved to {standard_time_data_file_path}")

Updated CSV file saved to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Standard_Time_Data_Source.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Extracting the corresponding raw data csv file sources to be processed.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Updating the 'Time_Series_Raw_Data_File_Path' of the 'standard_time_data_file_path' with the extracted raw data csv file sources paths.
</div>

In [12]:
def copy_and_organize_files(standard_time_data_file_path, standard_time_data_folder_path):
  """Copies and organizes CSV files based on information in a CSV file.

  Args:
    standard_time_data_file_path: Path to the CSV file containing file paths and codes.
    standard_time_data_folder_path: Path to the destination folder.
  """

  df = pd.read_csv(standard_time_data_file_path)

  for index, row in df.iterrows():
    source_file_path = row['Time_Series_Raw_Data_File_Path']
    dispa_set_code = row['Dispa-SET_Code']
    destination_folder = os.path.join(standard_time_data_folder_path, dispa_set_code)

    if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

    destination_file_path = os.path.join(destination_folder, os.path.basename(source_file_path))

    print(f"Copying file: {source_file_path} to {destination_file_path}")
    shutil.copy2(source_file_path, destination_file_path)

    df.at[index, 'Time_Series_Raw_Data_File_Path'] = destination_file_path

  df.to_csv(standard_time_data_file_path, index=False)

copy_and_organize_files(standard_time_data_file_path, standard_time_data_folder_path)

Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/AT/2024_1.csv to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/AT/2024_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/BE/2024_1.csv to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BE/2024_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/BG/2024_1.csv to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BG/2024_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/CH/2024_1.csv to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CH/2024_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/CY/2024_1.csv to /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CY/2024_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/Avail

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Writing to the 'standard_time_data_file_path' the path to the country folders that content all the data realted to the availability factor and scaled inflow time series in order to use them for next processing stages.
</div>

In [13]:
# Read the CSV file containing the paths
df = pd.read_csv(standard_time_data_file_path)

# Create a new column 'Zone_Folder_Path'
df['Zone_Folder_Path'] = df['Time_Series_Raw_Data_File_Path'].apply(lambda x: os.path.dirname(x))

# Save the updated DataFrame back to the CSV file
df.to_csv(standard_time_data_file_path, index=False)

print("Zone_Folder_Path column added successfully.")

Zone_Folder_Path column added successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    6. Raw Data Source File Processing
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Adding column names.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
All the raw data were downloaded without columns identification, the corresponding header to each column is added.
</div>

In [14]:
# Define the headers
headers = [
    "MTU", "Biomass_Actual_Aggregated", "Biomass_Actual_Consumption",
    "Fossil_Brown_coal-Lignite_Actual_Aggregated", "Fossil_Brown_coal-Lignite_Actual_Consumption",
    "Fossil_Coal-derived_gas_Actual_Aggregated", "Fossil_Coal-derived_gas_Actual_Consumption",
    "Fossil_Gas_Actual_Aggregated", "Fossil_Gas_Actual_Consumption",
    "Fossil_Hard_coal_Actual_Aggregated", "Fossil_Hard_coal_Actual_Consumption",
    "Fossil_Oil_Actual_Aggregated", "Fossil_Oil_Actual_Consumption",
    "Fossil_Oil_shale_Actual_Aggregated", "Fossil_Oil_shale_Actual_Consumption",
    "Fossil_Peat_Actual_Aggregated", "Fossil_Peat_Actual_Consumption",
    "Geothermal_Actual_Aggregated", "Geothermal_Actual_Consumption",
    "Hydro_Pumped_Storage_Actual_Aggregated", "Hydro_Pumped_Storage_Actual_Consumption",
    "Hydro_Run-of-river_and_poundage_Actual_Aggregated", "Hydro_Run-of-river_and_poundage_Actual_Consumption",
    "Hydro_Water_Reservoir_Actual_Aggregated", "Hydro_Water_Reservoir_Actual_Consumption",
    "Marine_Actual_Aggregated", "Marine_Actual_Consumption",
    "Nuclear_Actual_Aggregated", "Nuclear_Actual_Consumption",
    "Other_Actual_Aggregated", "Other_Actual_Consumption",
    "Other_renewable_Actual_Aggregated", "Other_renewable_Actual_Consumption",
    "Solar_Actual_Aggregated", "Solar_Actual_Consumption",
    "Waste_Actual_Aggregated", "Waste_Actual_Consumption",
    "Wind_Offshore_Actual_Aggregated", "Wind_Offshore_Actual_Consumption",
    "Wind_Onshore_Actual_Aggregated", "Wind_Onshore_Actual_Consumption"
]

# Read the CSV file containing the paths
df = pd.read_csv(standard_time_data_file_path)

success = True

# Iterate through each file path in the column 'Time_Series_Raw_Data_File_Path'
for index, row in df.iterrows():
    file_path = row['Time_Series_Raw_Data_File_Path']
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Open the existing CSV file in read mode
        with open(file_path, 'r', newline='') as file:
            # Read the existing content
            reader = csv.reader(file)
            rows = list(reader)

        # Insert a new empty row at the beginning
        rows.insert(0, [])

        # Write the updated content back to the CSV file
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rows)
        
        print(f"Empty row added at the beginning of file '{file_path}'.")

        # Read the file content again after adding the empty row
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Find the index of the first empty row
        empty_row_index = next((i for i, line in enumerate(lines) if line.strip() == ""), None)

        # If an empty row is found, copy the headers to it
        if empty_row_index is not None:
            lines[empty_row_index] = ','.join(headers) + '\n'

            # Write the updated lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(lines)
        else:
            print(f"No empty row found in file '{file_path}'. Headers not copied.")
    else:
        success = False
        print(f"File not found: {file_path}")

if success:
    print("Empty rows added and headers copied successfully to the first empty row of all files.")
else:
    print("Some errors occurred while processing the files.")

Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/AT/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BE/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BG/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CH/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CY/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CZ/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/DE/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/DK/2024_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/Availabili

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color: skyblue">
6.2. Raw Data Time Resolution
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Identifying the time spept of all downloaded files.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Additional columns that indicates the Year, Mounth, Day, Hour and minute of the data are added to the files.
</div>

In [15]:
# Read the CSV file containing the paths
df = pd.read_csv(standard_time_data_file_path)

# Iterate through each file path in the 'Time_Series_Raw_Data_File_Path' column
for file_path in df['Time_Series_Raw_Data_File_Path']:
    # Read the CSV file
    file_df = pd.read_csv(file_path)

    # Convert the 'MTU' column to string to avoid errors with .str accessor
    file_df['MTU'] = file_df['MTU'].astype(str)

    # Add new columns 'Year', 'Month', 'Day', and 'Time_1', 'Time_2'
    file_df['Year'] = data_year
    file_df['Month'] = ''
    file_df['Day'] = ''
    file_df['Hour'] = file_df['MTU'].str[:2]  # Extract the first two characters of 'MTU' column
    file_df['Minute'] = file_df['MTU'].str[3:5]  # Extract the third and fourth characters of 'MTU' column

    # Set the value of the first row in the 'Month' column to '01'
    # file_df.loc[0, 'Month'] = '01'
    # file_df.loc[0, 'Day'] = '01'

    # Reorder the columns to have 'Day' before 'Time'
    file_df = file_df[['Year', 'Month', 'Day', 'Hour', 'Minute'] + [col for col in file_df.columns if col not in ['Year', 'Month', 'Day', 'Hour', 'Minute']]]

    # Write the updated DataFrame back to the CSV file
    file_df.to_csv(file_path, index=False)

    print(f"Updated file: {file_path}")

print("All files updated successfully.")

Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/AT/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BE/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BG/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CH/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CY/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CZ/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/DE/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/DK/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/EE/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/EL/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/ES/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/FI/202

/tmp/ipykernel_1280822/3375416199.py:7: DtypeWarning: Columns (29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)
/tmp/ipykernel_1280822/3375416199.py:7: DtypeWarning: Columns (29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/PL/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/PT/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/RO/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SE/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SI/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SK/2024_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/UK/2024_1.csv
All files updated successfully.


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Extracting the time stept data.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The rows corresponding to a time resolution of 1 hour, 30 minutes and 15 minutes are extrated and added to new files with the suffix _1h, _30min, and _15min.
</div>

In [16]:
def process_csv_file(csv_file_path):
    print(f"Processing file: {csv_file_path}")
    
    # Load the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Count the number of rows
    num_rows = len(df)
    
    # Define file name without extension
    file_name_no_ext = os.path.splitext(os.path.basename(csv_file_path))[0]
    
    # Get the directory of the current CSV file
    base_dir = os.path.dirname(csv_file_path)
    
    # Check conditions and process accordingly
    if num_rows in [35040, 35136, 34544]:
        print(f"File has 35040 or 34544 rows. Processing...")
        process_35040_34544(df, file_name_no_ext, base_dir)
    elif num_rows in [17520, 17522, 17568, 17570]:
        print(f"File has 17520 or 17522 rows. Processing...")
        process_17520_17522(df, file_name_no_ext, base_dir)
    elif num_rows in [8760, 8761, 8784, 8785]:
        print(f"File has 8760 or 8761 rows. Processing...")
        process_8760_8761(df, file_name_no_ext, base_dir)
    else:
        print(f"File has {num_rows} rows. No specific conditions matched. Processing as default...")
        process_default(df, file_name_no_ext, base_dir)

def process_35040_34544(df, file_name_no_ext, base_dir):
    # Create new file paths
    suffixes = ['_1h', '_30min', '_15min']
    new_file_paths = [os.path.join(base_dir, file_name_no_ext + suffix + '.csv') for suffix in suffixes]
    
    # Write data to new files
    for new_file_path in new_file_paths:
        df.to_csv(new_file_path, index=False)
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    df_minute_zero_or_thirty = df[df['Minute'].isin([0, 30])]
    df_minute_zero_or_thirty.to_csv(os.path.join(base_dir, file_name_no_ext + '_30min.csv'), index=False)
    
    print("Processed successfully.")

def process_17520_17522(df, file_name_no_ext, base_dir):
    # Create new file paths
    suffixes = ['_1h', '_30min']
    new_file_paths = [os.path.join(base_dir, file_name_no_ext + suffix + '.csv') for suffix in suffixes]
    
    # Write data to new files
    for new_file_path in new_file_paths:
        df.to_csv(new_file_path, index=False)
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    print("Processed successfully.")

def process_8760_8761(df, file_name_no_ext, base_dir):
    # Create new file path
    new_file_path = os.path.join(base_dir, file_name_no_ext + '_1h.csv')
    
    # Write data to new file
    df.to_csv(new_file_path, index=False)
    
    print("Processed successfully.")

def process_default(df, file_name_no_ext, base_dir):
    # Create new file path
    new_file_path = os.path.join(base_dir, file_name_no_ext + '_1h.csv')
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    print("Processed successfully.")

# File path
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Load the CSV file
standard_time_data_df = pd.read_csv(standard_time_data_file_path)

# Iterate over each CSV file specified in 'Time_Series_Raw_Data_File_Path'
for csv_file_path in standard_time_data_df['Time_Series_Raw_Data_File_Path']:
    process_csv_file(csv_file_path)

Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/AT/2024_1.csv
File has 35040 or 34544 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BE/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BG/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CH/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CY/2024_1.csv
File has 0 rows. No specific conditions matched. Processing as default...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CZ/2024_1.csv
File has 0 rows. No specific conditions matched. Processing as default...
Processed successfully.
Processing file:

/tmp/ipykernel_1280822/1939109200.py:5: DtypeWarning: Columns (34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
/tmp/ipykernel_1280822/1939109200.py:5: DtypeWarning: Columns (34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/RO/2024_1.csv
File has 35040 or 34544 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SE/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SI/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/SK/2024_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/UK/2024_1.csv
File has 17520 or 17522 rows. Processing...
Processed successfully.


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filling the correspondig time stept.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
For the next stepts of the formating process it is necesary identify the path of the files where all the data of the 30 Dispa-SET countries are going to be storaged differencing them into time stepts (1 hour, 30 min, and 15 min)
</div>

In [17]:
# Define file paths and variables
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
#data_year = 2023

# Read the standard time data CSV file
df = pd.read_csv(standard_time_data_file_path)

# Define function to search for files in directory
def search_files(directory, prefix):
    for file in os.listdir(directory):
        if file.startswith(prefix):
            return os.path.join(directory, file)
    return None

# Add new columns
df[f"{data_year}_1h_File_Path"] = ""
df[f"{data_year}_30min_File_Path"] = ""
df[f"{data_year}_15min_File_Path"] = ""

# Iterate over Zone_Folder_Path column
for index, row in df.iterrows():
    zone_folder_path = row['Zone_Folder_Path']
    if os.path.exists(zone_folder_path):
        # Search for files in the directory
        hour_file_path = search_files(zone_folder_path, f"{data_year}_1_1h.csv")
        if hour_file_path:
            df.at[index, f"{data_year}_1h_File_Path"] = hour_file_path
            
        min30_file_path = search_files(zone_folder_path, f"{data_year}_1_30min.csv")
        if min30_file_path:
            df.at[index, f"{data_year}_30min_File_Path"] = min30_file_path
            
        min15_file_path = search_files(zone_folder_path, f"{data_year}_1_15min.csv")
        if min15_file_path:
            df.at[index, f"{data_year}_15min_File_Path"] = min15_file_path

# Save the updated DataFrame back to the same CSV file
df.to_csv(standard_time_data_file_path, index=False)

print("CSV file updated successfully.")

CSV file updated successfully.


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Fulling the corresponding time step to each file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Not all the countries have raw data in a resolution of 15 mimuntes, 30 minutes or 1 hour, so, all the next three cells all the files of the corresponding time stept are going to be fullfilling in their corresponding columns of Minute, Hour, Day and Month of the year specified in the variable data_year.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
<br>
1 Hour time stept.
</div>

In [18]:
# Specify the year
year = data_year

# Load the CSV file containing paths
df_paths = pd.read_csv(standard_time_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Get the path from the corresponding column
    file_path_column_name = f"{year}_1h_File_Path"  # Dynamically construct the column name based on the year
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year
        dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')

        # Extract month, day, and hour from the date range
        months = [date.month for date in dates]
        days = [date.day for date in dates]
        hours = [date.hour % 24 for date in dates]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data
        df['Month'] = months
        df['Day'] = days
        df['Hour'] = hours

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/AT/2024_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BE/2024_1_1h.csv


/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/BG/2024_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CH/2024_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CY/2024_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/CZ/2024_1_1h.csv


/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_1280822/3201936043.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


ValueError: Length of values (8784) does not match length of index (6480)

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
30 Minutes time stept.
</div>

In [ ]:
# Specify the year
year = data_year

# Load the CSV file containing paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(standard_time_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Dynamically construct the column name based on the year
    file_path_column_name = f"{year}_30min_File_Path"
    
    # Get the path from the corresponding column
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year with a time step of 30 minutes
        dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')

        # Extract month, day, and hour from the date range with 30-minute time step
        months_30min = [date.month for date in dates_30min]
        days_30min = [date.day for date in dates_30min]
        hours_30min = [date.hour % 24 for date in dates_30min]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data for 30-minute time step
        df['Month'] = months_30min
        df['Day'] = days_30min
        df['Hour'] = hours_30min

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully with 30-minute time step: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
15 Minutes time stept.
</div>

In [ ]:
# Specify the year
year = data_year

# Load the CSV file containing paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(standard_time_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Dynamically construct the column name based on the year
    file_path_column_name = f"{year}_15min_File_Path"
    
    # Get the path from the corresponding column
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year with a time step of 15 minutes
        dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')

        # Extract month, day, and hour from the date range with 15-minute time step
        months_15min = [date.month for date in dates_15min]
        days_15min = [date.day for date in dates_15min]
        hours_15min = [date.hour % 24 for date in dates_15min]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data for 15-minute time step
        df['Month'] = months_15min
        df['Day'] = days_15min
        df['Hour'] = hours_15min

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully with 15-minute time step: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Adding the Dispa-SET Time Stept format.
</div>
<div style="text-align: justify; margin-left: 1.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The format that Dispa-SET reads the time stept is the following, 0000-00-00 00:00:00+00:00 where the first part represents the date and the second part represents the time, so the next script add to each row of the correspongind file (1 hour file, 30 minutes file and 15 minutes file for each country) the time stept acording this format.
</div>

In [ ]:
# Define a function to create Dispa_SET_Time_Step column
def create_Dispa_SET_Time_Step(row):
    # Ensure values are integers (or default to 0 if NaN) before formatting
    year = int(row['Year']) if pd.notna(row['Year']) else 0
    month = int(row['Month']) if pd.notna(row['Month']) else 0
    day = int(row['Day']) if pd.notna(row['Day']) else 0
    hour = int(row['Hour']) if pd.notna(row['Hour']) else 0
    minute = int(row['Minute']) if pd.notna(row['Minute']) else 0
    
    # Ensure two-digit format for Day, Month, Hour, and Minute
    date_part = f"{year:04d}-{month:02d}-{day:02d}"
    time_part = f"{hour:02d}:{minute:02d}:00+00:00"
    return date_part + " " + time_part

# Specify the year
#data_year = 2023

# Load the CSV file containing file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(standard_time_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Get the file paths from the corresponding columns
    hour_file_path = row[f"{data_year}_1h_File_Path"]
    min_30_file_path = row[f"{data_year}_30min_File_Path"]
    min_15_file_path = row[f"{data_year}_15min_File_Path"]
    
    # Process each file path if it exists
    for file_path in [hour_file_path, min_30_file_path, min_15_file_path]:
        # Check if the path exists and is not NaN
        if isinstance(file_path, str) and os.path.exists(file_path):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Apply the function to create the Dispa_SET_Time_Step column
            df['Dispa_SET_Time_Step'] = df.apply(create_Dispa_SET_Time_Step, axis=1)
            
            # Save the modified DataFrame back to the CSV file
            df.to_csv(file_path, index=False)
            
            print(f"CSV file updated successfully: {file_path}")
        else:
            print(f"No valid path specified in row {index + 1}. Skipping...")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Erasing the unnecesary columns and ordering.
</div>

In [ ]:
# Define the function to process each CSV file
def process_csv_file(row, time_step_column):
    csv_file = row[time_step_column]
    if pd.notnull(csv_file) and os.path.exists(csv_file):
        # Read the CSV file into a DataFrame
        df_csv = pd.read_csv(csv_file)
        
        # Drop specified columns: Year, Month, Day, Hour, Minute, MTU
        columns_to_drop = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'MTU']
        df_csv = df_csv.drop(columns=columns_to_drop, errors='ignore')
        
        # Move the column 'Dispa_SET_Time_Step' to the first position
        if 'Dispa_SET_Time_Step' in df_csv.columns:
            columns = list(df_csv.columns)
            columns.remove('Dispa_SET_Time_Step')
            df_csv = df_csv[['Dispa_SET_Time_Step'] + columns]
        
        # Save the changes back to the CSV file
        df_csv.to_csv(csv_file, index=False)
        
# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file containing the paths
df = pd.read_csv(standard_time_data_file_path)

# Convert data_year to a string
data_year = str(data_year)

# Define the column names for the file paths
hour_file_column = data_year + '_1h_File_Path'
thirty_min_file_column = data_year + '_30min_File_Path'
fifteen_min_file_column = data_year + '_15min_File_Path'

# Process the one-hour file
df.apply(lambda row: process_csv_file(row, hour_file_column), axis=1)

# Process the thirty-minute file
df.apply(lambda row: process_csv_file(row, thirty_min_file_column), axis=1)

# Process the fifteen-minute file
df.apply(lambda row: process_csv_file(row, fifteen_min_file_column), axis=1)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6.3. Raw Data Subdirectories Creation
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the sub directories where the raw data is going to be splited by technology type.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Once the raw data file has the appropiaded time stept format, the same has to be divided to has the corresponding time serie of each technology type isolated.
<br>
A sub folder named as the corresponding technology type are goint to be created to each country.
</div>

In [ ]:
# Define the headers to be added
new_headers = [
    "Biomass_Actual_Aggregated", "Biomass_Actual_Consumption",
    "Fossil_Brown_coal-Lignite_Actual_Aggregated", "Fossil_Brown_coal-Lignite_Actual_Consumption",
    "Fossil_Coal-derived_gas_Actual_Aggregated", "Fossil_Coal-derived_gas_Actual_Consumption",
    "Fossil_Gas_Actual_Aggregated", "Fossil_Gas_Actual_Consumption",
    "Fossil_Hard_coal_Actual_Aggregated", "Fossil_Hard_coal_Actual_Consumption",
    "Fossil_Oil_Actual_Aggregated", "Fossil_Oil_Actual_Consumption",
    "Fossil_Oil_shale_Actual_Aggregated", "Fossil_Oil_shale_Actual_Consumption",
    "Fossil_Peat_Actual_Aggregated", "Fossil_Peat_Actual_Consumption",
    "Geothermal_Actual_Aggregated", "Geothermal_Actual_Consumption",
    "Hydro_Pumped_Storage_Actual_Aggregated", "Hydro_Pumped_Storage_Actual_Consumption",
    "Hydro_Run-of-river_and_poundage_Actual_Aggregated", "Hydro_Run-of-river_and_poundage_Actual_Consumption",
    "Hydro_Water_Reservoir_Actual_Aggregated", "Hydro_Water_Reservoir_Actual_Consumption",
    "Marine_Actual_Aggregated", "Marine_Actual_Consumption",
    "Nuclear_Actual_Aggregated", "Nuclear_Actual_Consumption",
    "Other_Actual_Aggregated", "Other_Actual_Consumption",
    "Other_renewable_Actual_Aggregated", "Other_renewable_Actual_Consumption",
    "Solar_Actual_Aggregated", "Solar_Actual_Consumption",
    "Waste_Actual_Aggregated", "Waste_Actual_Consumption",
    "Wind_Offshore_Actual_Aggregated", "Wind_Offshore_Actual_Consumption",
    "Wind_Onshore_Actual_Aggregated", "Wind_Onshore_Actual_Consumption"
]

# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file containing the paths
df = pd.read_csv(standard_time_data_file_path)

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract the zone folder path from the row
    zone_folder_path = row['Zone_Folder_Path']
    
    # Create a new folder for each zone folder path
    for header in new_headers:
        new_folder_path = os.path.join(zone_folder_path, header)
        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
        else:
            print(f"Folder '{new_folder_path}' already exists.")
            
        # Add the new column with the folder path to the DataFrame
        df.at[index, header] = new_folder_path

# Save the modified DataFrame back to the CSV file
df.to_csv(standard_time_data_file_path, index=False)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6.4. Raw Data Subdirectories Divition
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Splitting each raw data file of each country into a single file by technology type.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Each new csv file is goning to be saved in the corresponding folder created in the previous cell.
</div>

In [ ]:
# Path to the standard time data CSV file
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Load the standard time data CSV file
standard_time_df = pd.read_csv(standard_time_data_file_path)

# Get the data year from the variable
#data_year = "2023"  # Example value

# Iterate over each row in the standard time data
for index, row in standard_time_df.iterrows():
    # Extract the file paths from the specified columns
    hour_file_path = row[data_year + "_1h_File_Path"]
    thirty_min_file_path = row[data_year + "_30min_File_Path"]
    fifteen_min_file_path = row[data_year + "_15min_File_Path"]
    
    # Iterate over the file paths and apply the processing code
    for file_path in [hour_file_path, thirty_min_file_path, fifteen_min_file_path]:
        # Skip if the file path is empty
        if pd.isna(file_path):
            continue
        
        # Load the CSV file
        csv_df = pd.read_csv(file_path)
        
        # Get the headers
        headers = csv_df.columns
        
        # Create a folder for each second column
        for i in range(1, len(headers)):
            second_column_name = headers[i]
            folder_name = second_column_name.strip().replace(' ', '_')
            folder_path = os.path.join(os.path.dirname(file_path), folder_name)
            os.makedirs(folder_path, exist_ok=True)
        
        # Iterate over each pair of columns and save them into corresponding folders
        for i in range(len(headers) - 1):
            first_column = headers[0]
            second_column = headers[i + 1]
            new_df = csv_df[[first_column, second_column]]
            
            # Get the folder name for the second column
            folder_name = second_column.strip().replace(' ', '_')
            folder_path = os.path.join(os.path.dirname(file_path), folder_name)
            
            # Get the base file name without extension
            base_file_name = os.path.splitext(os.path.basename(file_path))[0]
            
            # Create the new file path
            new_file_path = os.path.join(folder_path, f"{base_file_name}.csv")
            
            # Save the new DataFrame to a CSV file inside the folder
            new_df.to_csv(new_file_path, index=False)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Replacing non numeric values in the separated files.
</div>

In [ ]:
def process_csv_files(new_headers, standard_time_data_file_path):
  """
  Processes CSV files, replacing non-numeric values in the second column with '0'.

  Args:
    new_headers: A list of column names.
    standard_time_data_file_path: Path to the CSV file containing folder paths.
  """

  df = pd.read_csv(standard_time_data_file_path)

  for column in new_headers:
    folder_paths = df[column]

    for path in folder_paths:
      if pd.notnull(path):
        csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

        for csv_file in csv_files:
          data = pd.read_csv(csv_file)
          data[data.columns[1]] = pd.to_numeric(data[data.columns[1]], errors='coerce').fillna(0)
          data.to_csv(csv_file, index=False)
          print(f"Processed {csv_file}")

process_csv_files(new_headers, standard_time_data_file_path)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
7. Total Installed Capacity per Production Type
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the files contents of the total installed capacity per production type for each one of the coutries modelled in Dispa-SET.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The source of the data belongs the ENTSO E portal web.
<br>
The information is extracted from the local directory:
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman; color:skyblue">
    '/Local/Directory/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Total_Installed_Capacity_per_Production_Type/'
</div>

In [ ]:
additional_path_3 = "Total_Installed_Capacity_per_Production_Type/"
total_instaled_capacity_per_production_type_folder_path = availability_factors_folder_path + additional_path_3

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [ ]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"standard_time:                                             {standard_time}")
print (f"standard_time_data_folder_path:                            {standard_time_data_folder_path}")
print (f"availability_factors_folder_path:                          {availability_factors_folder_path}")
print (f"scaled_outflows_folder_path:                               {scaled_outflows_folder_path}")
print (f"standard_time_data_file_path:                              {standard_time_data_file_path}")
print (f"standard_time_data_file_name:                              {standard_time_data_file_name}")
print (f"raw_rata_source_folder_path:                               {raw_rata_source_folder_path}")
print (f"total_instaled_capacity_per_production_type_folder_path:   {total_instaled_capacity_per_production_type_folder_path}")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Extracting and naming the total installed capacity raw data.
</div>

In [ ]:
def copy_and_rename_files(dispaSET_codes, availability_factors_folder_path, total_instaled_capacity_per_production_type_folder_path, data_year, standard_time_data_file_path):
    """
    Copies and renames CSV files based on specified parameters, and writes the paths to a new column in the standard time data CSV.

    Args:
        dispaSET_codes (list): A list of country codes.
        availability_factors_folder_path (str): Path to the destination folder.
        total_instaled_capacity_per_production_type_folder_path (str): Path to the source folder.
        data_year (str): The year to be included in the new file name.
        standard_time_data_file_path (str): Path to the standard time data CSV file.
    """
    # List to store the paths of the copied files
    copied_files_paths = []

    for code in dispaSET_codes:
        source_file = os.path.join(total_instaled_capacity_per_production_type_folder_path, code, 'Total_Installed_Capacity_per_Production_Type.csv')
        destination_folder = os.path.join(availability_factors_folder_path, code)
        destination_file = os.path.join(destination_folder, f"{data_year}_Total_Installed_Capacity_per_Production_Type.csv")

        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        shutil.copy2(source_file, destination_file)
        print(f"Copied file from {source_file} to {destination_file}")

        # Append the path of the copied file to the list
        copied_files_paths.append(destination_file)

        # Print a message indicating the file was processed
        print(f"Processed CSV file for {code}")

    # Read the standard time data CSV file
    standard_time_data = pd.read_csv(standard_time_data_file_path)

    # Add the new column with the copied files paths
    standard_time_data['Production_Type_Total_Installed_Capacity_File_Path'] = pd.Series(copied_files_paths)

    # Write the updated dataframe back to the CSV file
    standard_time_data.to_csv(standard_time_data_file_path, index=False)
    print(f"Updated {standard_time_data_file_path} with new file paths.")

# Example usage
# dispaSET_codes = ['CODE1', 'CODE2', 'CODE3']
# availability_factors_folder_path = '/path/to/availability_factors'
# total_instaled_capacity_per_production_type_folder_path = '/path/to/total_instaled_capacity'
# data_year = '2023'
# standard_time_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data_Source.csv'
copy_and_rename_files(dispaSET_codes, availability_factors_folder_path, total_instaled_capacity_per_production_type_folder_path, data_year, standard_time_data_file_path)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the corresponding headers to the columns of each downloaded file.
</div>

In [ ]:
# Function to add headers to CSV files
def add_headers_to_csv(csv_file_path):
    # Specify the headers
    headers = ['Production_Type', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    
    # Read the existing data from the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)

    # Add headers to the existing data
    rows.insert(0, headers)

    # Write the updated data back to the CSV file
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

# Function to process the standard time data
def process_standard_time_data(standard_time_data_file_path):
    # Open the standard time data file
    with open(standard_time_data_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Process each row in the standard time data
        for row in reader:
            # Get the path of the CSV file
            csv_file_path = row['Production_Type_Total_Installed_Capacity_File_Path']
            
            # Add headers to the CSV file
            add_headers_to_csv(csv_file_path)

# Specify the path to the CSV file containing the standard time data
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Process the standard time data
process_standard_time_data(standard_time_data_file_path)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Harmonizing the generation type column fields 
</div>

In [ ]:
# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file
df_paths = pd.read_csv(standard_time_data_file_path)

# Iterate over each row to process the files
for index, row in df_paths.iterrows():
    folder_path = row['Zone_Folder_Path']
    data_year = row['Data_Year']

    # Construct the file path
    file_name = f"{data_year}_Total_Installed_Capacity_per_Production_Type.csv"
    file_path = os.path.join(folder_path, file_name)

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Replace ' ' with '_' and '/' with '-'
    df['Production_Type'] = df['Production_Type'].str.replace(' ', '_').str.replace('/', '-')

    # Save the modified DataFrame back to the original CSV file
    df.to_csv(file_path, index=False)

    # Print a confirmation message for each file
    print(f"Production types in {file_name} have been processed and saved back to the original file.")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the paths to each total installed capacity file for each country.
</div>

In [ ]:
# Define variables
#data_year = 2024
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the standard time data CSV file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Function to get the file path for each row
def get_csv_file_path(row):
    zone_folder_path = row['Zone_Folder_Path']
    csv_file_name = f"{data_year}_Total_Installed_Capacity_per_Production_Type.csv"
    csv_file_path = os.path.join(zone_folder_path, csv_file_name)
    return csv_file_path

# Add a new column 'Production_Type_Total_Installed_Capacity_File_Path' with file paths
standard_time_data['Production_Type_Total_Installed_Capacity_File_Path'] = standard_time_data.apply(get_csv_file_path, axis=1)

# Save the modified DataFrame back to the CSV file
standard_time_data.to_csv(standard_time_data_file_path, index=False)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Fullfilling missing years to each total installed capacity file for each country.
</div>

In [ ]:
# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Iterate over each file path in the 'Production_Type_Total_Installed_Capacity_file_path' column
for index, row in standard_time_data.iterrows():
    # Get the file path
    csv_file_path = row['Production_Type_Total_Installed_Capacity_File_Path']
    
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    # Iterate over columns from '2015' to '2024'
    for column in df.columns[1:]:
        # Iterate over rows in the column
        for index, value in df[column].items():
            # Check if the value is non-numeric or empty
            if str(value).strip() in ('n/e', 'N/A', '', '0'):
                # Get the corresponding value from the previous column
                prev_value = df.at[index, df.columns[df.columns.get_loc(column) - 1]]
                # Check if the previous value is numeric
                if str(prev_value).strip().replace('.', '').isdigit():
                    # Copy the previous value to the current field
                    df.at[index, column] = prev_value

    # Save the modified DataFrame back to the CSV file
    df.to_csv(csv_file_path, index=False)

# Print a completion message
print("Missing or non-numeric values in the CSV files have been filled based on the corresponding values from the previous columns where applicable.")

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filling the empty or 'n/e' or 'N/A' fields.
<br> 

In [ ]:
def fill_missing_values(standard_time_data_file_path):
    """Fills missing values in CSV files with '0'.

    Args:
        standard_time_data_file_path (str): Path to the standard time data CSV file.
    """

    df = pd.read_csv(standard_time_data_file_path)

    for index, row in df.iterrows():
        file_path = row['Production_Type_Total_Installed_Capacity_File_Path']
        data = pd.read_csv(file_path)
        data.fillna(0, inplace=True)
        data = data.replace('n/e', 0)
        data = data.replace('N/A', 0)
        data.to_csv(file_path, index=False)
        print(f"Processed file: {file_path}")

fill_missing_values(standard_time_data_file_path)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
8. Availability Factor File
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the time series value per technology type.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
All the raw data files  of generation and already separated by technology type is divided by the total installed capacity of the corresponding technology.
</div>

In [ ]:
# Define variables
# Convert data_year to a string
data_year = str(data_year)
#standard_time_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv'

# Read the CSV file containing file paths
df_production_type = pd.read_csv(standard_time_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_production_type.iterrows():
    # Extract the file path for the current production type
    file_path = row['Production_Type_Total_Installed_Capacity_File_Path']
    
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert 'data_year' column to numeric type
    df[data_year] = pd.to_numeric(df[data_year], errors='coerce').fillna(0)

    # Iterate over each row
    for index, row in df.iterrows():
        value = row[data_year]
        production_type = row['Production_Type']

        # Find the corresponding folder
        folder_name = production_type.replace(' ', '_')
        aggregated_folder = os.path.join(os.path.dirname(file_path), f"{folder_name}_Actual_Aggregated")
        consumption_folder = os.path.join(os.path.dirname(file_path), f"{folder_name}_Actual_Consumption")

        # Process files in aggregated folder
        if os.path.exists(aggregated_folder):
            for filename in os.listdir(aggregated_folder):
                if filename.endswith('.csv'):
                    file_path_aggregated = os.path.join(aggregated_folder, filename)
                    df_aggregated = pd.read_csv(file_path_aggregated)

                    # Convert second column to numeric type
                    df_aggregated.iloc[:, 1] = pd.to_numeric(df_aggregated.iloc[:, 1], errors='coerce').fillna(0)

                    # Check if value is not 'n/e', 'N/A', 0, or empty
                    if value not in ['n/e', 'N/A', 0, '']:
                        # Perform division operation
                        if value != 0:
                            df_aggregated.iloc[:, 1] /= value
                        else:
                            # Handle division by zero
                            pass
                    else:
                        # Fill second column with 0
                        df_aggregated.iloc[:, 1] = 0

                    new_file_name = f"Availabilty_Factor_{filename}"
                    df_aggregated.to_csv(os.path.join(aggregated_folder, new_file_name), index=False)

        # Process files in consumption folder
        if os.path.exists(consumption_folder):
            for filename in os.listdir(consumption_folder):
                if filename.endswith('.csv'):
                    file_path_consumption = os.path.join(consumption_folder, filename)
                    df_consumption = pd.read_csv(file_path_consumption)

                    # Convert second column to numeric type
                    df_consumption.iloc[:, 1] = pd.to_numeric(df_consumption.iloc[:, 1], errors='coerce').fillna(0)

                    # Check if value is not 'n/e', 'N/A', 0, or empty
                    if value not in ['n/e', 'N/A', 0, '']:
                        # Perform division operation
                        if value != 0:
                            df_consumption.iloc[:, 1] /= value
                        else:
                            # Handle division by zero
                            pass
                    else:
                        # Fill second column with 0
                        df_consumption.iloc[:, 1] = 0

                    new_file_name = f"Availabilty_Factor_{filename}"
                    df_consumption.to_csv(os.path.join(consumption_folder, new_file_name), index=False)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the subfolders and the csv files that will content the final data.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Into each country folder, it will be created a subfolder named under the corresponding time step of the data. e.g. '1h', and '15min' or '30min' being possible that some countries has the 3 files or just 1 or 2. 
<br>
This is due to the availability of the data in the sources.
</div>

In [ ]:
# Define variables
#standard_time_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv'
#data_year = '2023'
path_columns = ['Zone_Folder_Path', 'Biomass_Actual_Aggregated', 'Biomass_Actual_Consumption', 'Fossil_Brown_coal-Lignite_Actual_Aggregated',
                'Fossil_Brown_coal-Lignite_Actual_Consumption', 'Fossil_Coal-derived_gas_Actual_Aggregated', 'Fossil_Coal-derived_gas_Actual_Consumption',
                'Fossil_Gas_Actual_Aggregated',	'Fossil_Gas_Actual_Consumption', 'Fossil_Hard_coal_Actual_Aggregated', 'Fossil_Hard_coal_Actual_Consumption',
                'Fossil_Oil_Actual_Aggregated',	'Fossil_Oil_Actual_Consumption', 'Fossil_Oil_shale_Actual_Aggregated', 'Fossil_Oil_shale_Actual_Consumption',
                'Fossil_Peat_Actual_Aggregated', 'Fossil_Peat_Actual_Consumption', 'Geothermal_Actual_Aggregated', 'Geothermal_Actual_Consumption', 
                'Hydro_Pumped_Storage_Actual_Aggregated', 'Hydro_Pumped_Storage_Actual_Consumption', 'Hydro_Run-of-river_and_poundage_Actual_Aggregated',
                'Hydro_Run-of-river_and_poundage_Actual_Consumption', 'Hydro_Water_Reservoir_Actual_Aggregated', 'Hydro_Water_Reservoir_Actual_Consumption',
                'Marine_Actual_Aggregated',	'Marine_Actual_Consumption', 'Nuclear_Actual_Aggregated', 'Nuclear_Actual_Consumption', 'Other_Actual_Aggregated',
                'Other_Actual_Consumption',	'Other_renewable_Actual_Aggregated', 'Other_renewable_Actual_Consumption', 'Solar_Actual_Aggregated',
                'Solar_Actual_Consumption',	'Waste_Actual_Aggregated', 'Waste_Actual_Consumption', 'Wind_Offshore_Actual_Aggregated', 'Wind_Offshore_Actual_Consumption',
                'Wind_Onshore_Actual_Aggregated', 'Wind_Onshore_Actual_Consumption'
    
]  # Replace with your column names

# Read the standard time data CSV file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Iterate over each path column
for column in path_columns:
    # Iterate over each row in the DataFrame
    for index, row in standard_time_data.iterrows():
        # Extract the folder path
        folder_path = row[column]
        
        # Check if the folder exists
        if os.path.exists(folder_path):
            # Count the number of existing CSV files in the folder
            existing_files = []
            for time_interval in ['1h', '15min', '30min']:
                file_name = f"{data_year}_1_{time_interval}.csv"
                file_path = os.path.join(folder_path, file_name)
                if os.path.exists(file_path):
                    existing_files.append(time_interval)
            
            # Create subfolders based on the number of existing files
            for time_interval in existing_files:
                subfolder_path = os.path.join(folder_path, time_interval)
                os.makedirs(subfolder_path, exist_ok=True)
                
                # Create a CSV file inside the subfolder with the name of data_year
                csv_file_path = os.path.join(subfolder_path, f"{data_year}.csv")
                with open(csv_file_path, 'w') as f:
                    f.write("This is a sample CSV file.")
                    # Print a message indicating the creation of the file
                print(f"File '{data_year}.csv' created in '{subfolder_path}'")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the data processed to a separated file accordingly the technology type.
</div>

In [ ]:
def copy_csv_content(data_year, standard_time_data_file_path):
    """
    Copy content from availability factor CSV files to destination CSV files based on specified conditions.

    Args:
        data_year (int): The year integer.
        standard_time_data_file_path (str): Path to the standard time data CSV file.
    """
    # Read the standard time data CSV file
    df_standard_time = pd.read_csv(standard_time_data_file_path)

    # Define the columns to process
    columns_to_process = [
        'Biomass_Actual_Aggregated', 'Biomass_Actual_Consumption',
        'Fossil_Brown_coal-Lignite_Actual_Aggregated', 'Fossil_Brown_coal-Lignite_Actual_Consumption',
        'Fossil_Coal-derived_gas_Actual_Aggregated', 'Fossil_Coal-derived_gas_Actual_Consumption',
        'Fossil_Gas_Actual_Aggregated', 'Fossil_Gas_Actual_Consumption',
        'Fossil_Hard_coal_Actual_Aggregated', 'Fossil_Hard_coal_Actual_Consumption',
        'Fossil_Oil_Actual_Aggregated', 'Fossil_Oil_Actual_Consumption',
        'Fossil_Oil_shale_Actual_Aggregated', 'Fossil_Oil_shale_Actual_Consumption',
        'Fossil_Peat_Actual_Aggregated', 'Fossil_Peat_Actual_Consumption',
        'Geothermal_Actual_Aggregated', 'Geothermal_Actual_Consumption',
        'Hydro_Pumped_Storage_Actual_Aggregated', 'Hydro_Pumped_Storage_Actual_Consumption',
        'Hydro_Run-of-river_and_poundage_Actual_Aggregated', 'Hydro_Run-of-river_and_poundage_Actual_Consumption',
        'Hydro_Water_Reservoir_Actual_Aggregated', 'Hydro_Water_Reservoir_Actual_Consumption',
        'Marine_Actual_Aggregated', 'Marine_Actual_Consumption',
        'Nuclear_Actual_Aggregated', 'Nuclear_Actual_Consumption',
        'Other_Actual_Aggregated', 'Other_Actual_Consumption',
        'Other_renewable_Actual_Aggregated', 'Other_renewable_Actual_Consumption',
        'Solar_Actual_Aggregated', 'Solar_Actual_Consumption',
        'Waste_Actual_Aggregated', 'Waste_Actual_Consumption',
        'Wind_Offshore_Actual_Aggregated', 'Wind_Offshore_Actual_Consumption',
        'Wind_Onshore_Actual_Aggregated', 'Wind_Onshore_Actual_Consumption'
    ]

    # Define the subfolder mapping
    subfolder_mapping = {
        '1h': f'Availabilty_Factor_{data_year}_1_1h.csv',
        '15min': f'Availabilty_Factor_{data_year}_1_15min.csv',
        '30min': f'Availabilty_Factor_{data_year}_1_30min.csv'
    }

    # Iterate over each row in the dataframe
    for index, row in df_standard_time.iterrows():
        for column in columns_to_process:
            if column in row:
                base_path = row[column]

                # Iterate over each subfolder
                for subfolder, source_file_name in subfolder_mapping.items():
                    source_file_path = os.path.join(base_path, source_file_name)
                    dest_folder_path = os.path.join(base_path, subfolder)
                    dest_file_path = os.path.join(dest_folder_path, f'{data_year}.csv')

                    # Check if the source CSV file exists
                    if os.path.exists(source_file_path):
                        # Ensure the destination subfolder exists
                        os.makedirs(dest_folder_path, exist_ok=True)

                        # Copy content from source to destination file, overwriting if exists
                        shutil.copy2(source_file_path, dest_file_path)
                        print(f"Copied from '{source_file_path}' to '{dest_file_path}'")
                    else:
                        print(f"Source file does not exist: '{source_file_path}'")

copy_csv_content(data_year, standard_time_data_file_path)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the corresponding headers to the csv files that will content the final data.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The Availability Factor Time Series have a determined header according the renewable techology type (HROR, PHOT, WTON, WOTF) , those are used to be read by Dispa-SET.
</div>

In [ ]:
def replace_headers(data_year, standard_time_data_file_path):
    """
    Replace headers in CSV files based on specified conditions without erasing the existing data.

    Args:
        data_year (str): The year string.
        standard_time_data_file_path (str): Path to the standard time data CSV file.
    """
    # Read the standard time data CSV file
    standard_time_data = pd.read_csv(standard_time_data_file_path)

    # Define header mappings
    header_mapping = {
        'Hydro_Run-of-river_and_poundage_Actual_Aggregated': ['', 'HROR'],
        'Solar_Actual_Aggregated': ['', 'PHOT'],
        'Wind_Offshore_Actual_Aggregated': ['', 'WTOF'],
        'Wind_Onshore_Actual_Aggregated': [' ', 'WTON']
    }

    # Iterate over each column in the dataframe
    for column in standard_time_data.columns:
        if column in header_mapping:
            # Iterate over each folder path in the column
            for path in standard_time_data[column]:
                # Iterate over each subfolder
                for subfolder in ['1h', '15min', '30min']:
                    # Construct the path to the CSV file
                    csv_file_path = os.path.join(path, subfolder, f'{data_year}.csv')
                    
                    # Check if the CSV file exists
                    if os.path.exists(csv_file_path):
                        # Read the existing data from the CSV file
                        df = pd.read_csv(csv_file_path)
                        
                        # Define the new headers
                        headers = header_mapping[column]
                        # Update the headers of the DataFrame
                        new_headers = headers + list(df.columns[len(headers):])
                        df.columns = new_headers
                        
                        # Write the DataFrame with new headers back to the CSV file
                        df.to_csv(csv_file_path, index=False)
                        
                        print(f"Headers replaced for '{csv_file_path}'")

replace_headers(data_year, standard_time_data_file_path)

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Controling the range (min = 0, max = 1) of the time series.
<div/>

In [ ]:
# Read the standard time data CSV file
df = pd.read_csv(standard_time_data_file_path)

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    solar_path = row['Solar_Actual_Aggregated']
    
    # Define the subfolders we are interested in
    subfolders = ['1h', '15min', '30min']
    
    for subfolder in subfolders:
        # Construct the path to the subfolder's CSV file
        csv_file_path = os.path.join(solar_path, subfolder, f"{data_year}.csv")
        
        # Check if the CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file
            subfolder_df = pd.read_csv(csv_file_path)
            
            # Check for any value in the 'PHOT' column that is greater than 1 and set it to 1
            subfolder_df['PHOT'] = subfolder_df['PHOT'].apply(lambda x: 1 if x > 1 else x)
            
            # Save the updated CSV file
            subfolder_df.to_csv(csv_file_path, index=False)
            print(f"Updated {csv_file_path} where PHOT values were greater than 1.")
        else:
            print(f"CSV file {csv_file_path} does not exist.")

print("All files have been processed.")

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
8.2 Final Availability Factor Time Series
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the Availability Factor Time Series file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
All the data is copied to the single csv file in a sub-folder with the time stept inside the country folder (AT, BE, CH... etc) .
</div>

In [ ]:
#data_year = "2023"
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file to get the source file paths
df_standard_time = pd.read_csv(standard_time_data_file_path)

# Iterate over each row of the specified columns
for index, row in df_standard_time.iterrows():
    # Extract the source file paths from the current row
    file_path_1 = row['Zone_Folder_Path']
    file_path_2 = row['Wind_Onshore_Actual_Aggregated']
    file_path_3 = row['Wind_Offshore_Actual_Aggregated']
    file_path_4 = row['Solar_Actual_Aggregated']
    file_path_5 = row['Hydro_Run-of-river_and_poundage_Actual_Aggregated']
    
    # Define the subfolders to iterate over
    subfolders = ['1h', '15min', '30min']
    
    for subfolder in subfolders:
        # Construct the file paths for each subfolder
        csv_file_path_1 = os.path.join(file_path_1, subfolder, f"{data_year}.csv")
        csv_file_path_2 = os.path.join(file_path_2, subfolder, f"{data_year}.csv")
        csv_file_path_3 = os.path.join(file_path_3, subfolder, f"{data_year}.csv")
        csv_file_path_4 = os.path.join(file_path_4, subfolder, f"{data_year}.csv")
        csv_file_path_5 = os.path.join(file_path_5, subfolder, f"{data_year}.csv")
        
        # Check if CSV files exist in file_path_2, file_path_3, file_path_4, and file_path_5
        if os.path.exists(csv_file_path_2) and os.path.exists(csv_file_path_3) \
            and os.path.exists(csv_file_path_4) and os.path.exists(csv_file_path_5):
            
            # Read the contents of the CSV files
            df_2 = pd.read_csv(csv_file_path_2)
            df_3 = pd.read_csv(csv_file_path_3)
            df_4 = pd.read_csv(csv_file_path_4)
            df_5 = pd.read_csv(csv_file_path_5)
            
            # Combine the contents of the CSV files
            combined_df = pd.concat([df_2.iloc[:, :2], df_3.iloc[:, 1], df_4.iloc[:, 1], df_5.iloc[:, 1]], axis=1)
            
            # Delete the destination CSV file if it exists
            if os.path.exists(csv_file_path_1):
                os.remove(csv_file_path_1)
            
            # Write the combined DataFrame to the CSV file in file_path_1
            combined_df.to_csv(csv_file_path_1, index=False)
            
            print(f"Contents from CSV files written to '{csv_file_path_1}'")
        else:
            print(f"One or more CSV files do not exist at the specified paths")

In [ ]:
# Define the path to the CSV file
#standard_time_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv'

# Read the CSV file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Assuming data_year is already defined
#data_year = "2023"

# Columns to check
columns_to_check = ['WTON', 'WTOF', 'PHOT', 'HROR']

# List of subfolders to check
subfolders = ['1h', '15min', '30min']

# Iterate over each row in the DataFrame
for index, row in standard_time_data.iterrows():
    # Get the folder path from the 'Zone_Folder_Path' column
    folder_path = row['Zone_Folder_Path']
    
    # Iterate over subfolders
    for subfolder in subfolders:
        csv_file_path = os.path.join(folder_path, subfolder, f"{data_year}.csv")
        
        # Check if CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file
            df = pd.read_csv(csv_file_path)
            
            # Iterate over specified columns
            for column in columns_to_check:
                # Convert column values to numeric
                df[column] = pd.to_numeric(df[column], errors='coerce')
                
                # Check if the value is greater than 1
                df.loc[df[column] > 1, column] = 1
            
            # Write the modified DataFrame back to the CSV file
            df.to_csv(csv_file_path, index=False)
            
            print(f"Values in CSV file '{csv_file_path}' modified successfully")
        else:
            print(f"CSV file '{csv_file_path}' not found")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
The final Availability Factor Time Series is for each Country modeled in Dispa-SET is located in the following local directory:
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
'/Local/Path/to/Dispas-SET/RawData/AvailabiltyFactors/'
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman; color:skyblue">
Inside this path there are folders with the acronym of each country modelled in Dipsa-SET. i.e. AT, BE, CH.... UK
<br>
Inside each of this folders, there are sub folders named by the time stept of the time series. i.e. 1h, 30min and/or 15min.
<br>
Inside these sub folders, it is going to be found the corresponding time series .csv file named with the year of the data. e.g. 2023.csv
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
8.3 Availability Factor Files Final Location
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the already processed data to the Dispa-SET data base folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
'/Local/Path/to/Dispas-SET/Database/AvailabiltyFactors/'
</div>

In [ ]:
additional_path_4 = "/Database/AvailabilityFactors/"

# Construct the power_plants_raw_data_folder_path variable
availability_factor_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_4

In [ ]:
def copy_csv_files(data_year, dispaSET_codes, availability_factors_folder_path, availability_factor_data_base_folder_path):
    """Copies CSV files from source to destination, replacing existing files if necessary.

    Args:
        data_year (int): The year used in the file names.
        dispaSET_codes (list): List of country codes corresponding to folders.
        availability_factors_folder_path (str): Path to the source directory.
        availability_factor_data_base_folder_path (str): Path to the destination directory.
    """
    
    # Iterate over each country code
    for code in dispaSET_codes:
        source_folder = os.path.join(availability_factors_folder_path, code)
        destination_folder = os.path.join(availability_factor_data_base_folder_path, code)

        # Iterate over each subfolder (1h, 15min, 30min)
        for subfolder in ['1h', '15min', '30min']:
            source_subfolder = os.path.join(source_folder, subfolder)
            destination_subfolder = os.path.join(destination_folder, subfolder)
            
            # Check if the source subfolder exists
            if os.path.exists(source_subfolder):
                # Ensure the destination subfolder exists, create it if it doesn't
                if not os.path.exists(destination_subfolder):
                    os.makedirs(destination_subfolder)
                
                # Define the CSV file name
                csv_file_name = f"{data_year}.csv"
                
                source_file = os.path.join(source_subfolder, csv_file_name)
                destination_file = os.path.join(destination_subfolder, csv_file_name)
                
                # Check if the source file exists
                if os.path.exists(source_file):
                    # Copy the file to the destination, replacing it if it exists
                    shutil.copy2(source_file, destination_file)
                    print(f"Copied file: {source_file} to {destination_file}")
                else:
                    print(f"Source file does not exist: {source_file}")
            else:
                print(f"Source subfolder does not exist: {source_subfolder}")

# Call the function to perform the copying
copy_csv_files(data_year, dispaSET_codes, availability_factors_folder_path, availability_factor_data_base_folder_path)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
9. Scaled Outflows File
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the content folders for the scaled outflows data for each country.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The created folders are storaged in the directory.
</div>
<div style="text-align: justify; margin-left: 3.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
/local/path/to/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/ScaledOutflows
</div>

In [ ]:
# Read the standard time data file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Create a new column 'Scaled_outflow_Folder_Path' with default values
standard_time_data['Scaled_Outflow_Folder_Path'] = ''

# Create a new column 'Availability_Factors_Folder_Path' with default values
standard_time_data['Availability_Factors_Folder_Path'] = ''

# Iterate over each row in the DataFrame
for index, row in standard_time_data.iterrows():
    # Get the value of the 'Dispa-SET_Code' column
    dispa_set_code = row['Dispa-SET_Code']
    
    # Create the scaled outflows folder path
    scaled_outflows_path = os.path.join(scaled_outflows_folder_path, dispa_set_code)
    
    # Create the availability factors folder path
    availability_factors_path = os.path.join(availability_factors_folder_path, dispa_set_code)
    
    # Create the folder for scaled outflows if it doesn't exist
    if not os.path.exists(scaled_outflows_path):
        os.makedirs(scaled_outflows_path)
        print(f"Scaled outflows folder created: {scaled_outflows_path}")
    
    # Update the 'Scaled_Outflow_Folder_Path' column with the folder path
    standard_time_data.at[index, 'Scaled_Outflow_Folder_Path'] = scaled_outflows_path
    
    # Update the 'Availability_Factors_Folder_Path' column with the availability factors path
    standard_time_data.at[index, 'Availability_Factors_Folder_Path'] = availability_factors_path

# Save the modified DataFrame back to the CSV file
standard_time_data.to_csv(standard_time_data_file_path, index=False)

print("Folder paths added to the CSV file.")

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv(standard_time_data_file_path)

# Function to extract folders from a path
def extract_folders(path):
    if pd.notnull(path):
        return set(os.listdir(path))
    else:
        return set()

# Function to create folders for each path in a given column
def create_folders_for_paths(row):
    availability_folders = extract_folders(row['Availability_Factors_Folder_Path'])
    for folder in ['1h', '15min', '30min']:
        if folder in availability_folders:
            folder_path = os.path.join(row['Scaled_Outflow_Folder_Path'], folder)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
                print(f"Created folder: {folder_path}")

# Apply the function to each row
df.apply(create_folders_for_paths, axis=1)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the corresponding headers to the csv files that will content the final data.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The Scaled Inflows Time Series have a determined header according the techology type (HPHS, HDAM), those are used to be read by Dispa-SET.
</div>

In [ ]:
def replace_headers(data_year, standard_time_data_file_path):
    """
    Replace headers in CSV files based on specified conditions without erasing the existing data.

    Args:
        data_year (str): The year string.
        standard_time_data_file_path (str): Path to the standard time data CSV file.
    """
    # Read the standard time data CSV file
    standard_time_data = pd.read_csv(standard_time_data_file_path)

    # Define header mappings
    header_mapping = {
        'Hydro_Pumped_Storage_Actual_Aggregated': ['', 'HPHS'],
        'Hydro_Water_Reservoir_Actual_Aggregated': ['', 'HDAM']

    }

    # Iterate over each column in the dataframe
    for column in standard_time_data.columns:
        if column in header_mapping:
            # Iterate over each folder path in the column
            for path in standard_time_data[column]:
                # Iterate over each subfolder
                for subfolder in ['1h', '15min', '30min']:
                    # Construct the path to the CSV file
                    csv_file_path = os.path.join(path, subfolder, f'{data_year}.csv')
                    
                    # Check if the CSV file exists
                    if os.path.exists(csv_file_path):
                        # Read the existing data from the CSV file
                        df = pd.read_csv(csv_file_path)
                        
                        # Define the new headers
                        headers = header_mapping[column]
                        # Update the headers of the DataFrame
                        new_headers = headers + list(df.columns[len(headers):])
                        df.columns = new_headers
                        
                        # Write the DataFrame with new headers back to the CSV file
                        df.to_csv(csv_file_path, index=False)
                        
                        print(f"Headers replaced for '{csv_file_path}'")

replace_headers(data_year, standard_time_data_file_path)

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the final Scaled Ouflows Time Series file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
All the data is copied to the single csv file in a sub-folder with the time stept inside the country folder (AT, BE, CH... etc) .
</div>

In [ ]:
#data_year = "2023"
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file to get the source file paths
df_standard_time = pd.read_csv(standard_time_data_file_path)

# Iterate over each row of the specified columns
for index, row in df_standard_time.iterrows():
    # Extract the source file paths from the current row
    file_path_1 = row['Scaled_Outflow_Folder_Path']
    file_path_2 = row['Hydro_Pumped_Storage_Actual_Aggregated']
    file_path_3 = row['Hydro_Water_Reservoir_Actual_Aggregated']
    
    # Define the subfolders to iterate over
    subfolders = ['1h', '15min', '30min']
    
    for subfolder in subfolders:
        # Construct the file paths for each subfolder
        csv_file_path_1 = os.path.join(file_path_1, subfolder, f"{data_year}.csv")
        csv_file_path_2 = os.path.join(file_path_2, subfolder, f"{data_year}.csv")
        csv_file_path_3 = os.path.join(file_path_3, subfolder, f"{data_year}.csv")
        
        # Check if CSV files exist in file_path_2, file_path_3
        if os.path.exists(csv_file_path_2) and os.path.exists(csv_file_path_3):
            
            # Read the contents of the CSV files
            df_2 = pd.read_csv(csv_file_path_2)
            df_3 = pd.read_csv(csv_file_path_3)
           
            
            # Combine the contents of the CSV files
            combined_df = pd.concat([df_2.iloc[:, :2], df_3.iloc[:, 1]], axis=1)
            
            # Delete the destination CSV file if it exists
            if os.path.exists(csv_file_path_1):
                os.remove(csv_file_path_1)
            
            # Write the combined DataFrame to the CSV file in file_path_1
            combined_df.to_csv(csv_file_path_1, index=False)
            
            print(f"Contents from CSV files written to '{csv_file_path_1}'")
        else:
            print(f"One or more CSV files do not exist at the specified paths")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Verifying the scaled outflow factor value.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since the factor value has to be between 0 and 1, the same has to be verified to be in the range.
</div>

In [ ]:
# Define the path to the CSV file
#standard_time_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv'

# Read the CSV file
standard_time_data = pd.read_csv(standard_time_data_file_path)

# Assuming data_year is already defined
#data_year = "2023"

# Columns to check
columns_to_check = ['HPHS', 'HDAM']

# List of subfolders to check
subfolders = ['1h', '15min', '30min']

# Iterate over each row in the DataFrame
for index, row in standard_time_data.iterrows():
    # Get the folder path from the 'Zone_Folder_Path' column
    folder_path = row['Scaled_Outflow_Folder_Path']
    
    # Iterate over subfolders
    for subfolder in subfolders:
        csv_file_path = os.path.join(folder_path, subfolder, f"{data_year}.csv")
        
        # Check if CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file
            df = pd.read_csv(csv_file_path)
            
            # Iterate over specified columns
            for column in columns_to_check:
                # Convert column values to numeric
                df[column] = pd.to_numeric(df[column], errors='coerce')
                
                # Check if the value is greater than 1
                df.loc[df[column] > 1, column] = 1
            
            # Write the modified DataFrame back to the CSV file
            df.to_csv(csv_file_path, index=False)
            
            print(f"Values in CSV file '{csv_file_path}' modified successfully")
        else:
            print(f"CSV file '{csv_file_path}' not found")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
The final Scaled Outflows Time Series is for each Country modeled in Dispa-SET is located in the following local directory:
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
'/Local/Path/to/Dispas-SET/RawData/HydroData/ScaledInflows/ScaledOutFlows/'
<br>
Inside this path there are folders with the acronym of each country modelled in Dipsa-SET. i.e. AT, BE, CH.... UK
<br>
Inside each of this folders, there are sub folders named by the time stept of the time series. i.e. 1h, 30min and/or 15min.
<br>
Inside these sub folders, it is going to be found the corresponding time series .csv file named with the year of the data. e.g. 2023.csv
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Additionally, the total installed capacity per production type file is also requiered to format the scaled inflows time series.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The corresponding files that content this data are going to be copied to the scalled inflows folders.
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Defining the destination folders to the total hydro installed capacity data.
</div>

In [ ]:
additional_path_7 = "/RawData/HydroData/ScaledInflows/Total_Hydro_Installed_Capacity_per_Production_Type/"

# Construct the total_hydro_installed_capacity_folder_path variable
total_hydro_installed_capacity_folder_path = dispaSET_unleash_folder_path + additional_path_7

In [ ]:
total_hydro_installed_capacity_folder_path

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the needed data to the destination folders.
</div>

In [ ]:
def extract_hydro_data(dispaSET_codes, availability_factors_folder_path, total_hydro_installed_capacity_folder_path, data_year):
  """
  Extracts hydro data from CSV files and saves to new files.

  Args:
    dispaSET_codes: A list of dispaSET codes.
    availability_factors_folder_path: The path to the availability factors folder.
    total_hydro_installed_capacity_folder_path: The path to the output folder.
    data_year: The data year.
  """

  for code in dispaSET_codes:
    input_file = os.path.join(availability_factors_folder_path, code, f"{data_year}_Total_Installed_Capacity_per_Production_Type.csv")
    output_folder = os.path.join(total_hydro_installed_capacity_folder_path, code)

    if not os.path.exists(output_folder):
      os.makedirs(output_folder)

    df = pd.read_csv(input_file)
    hydro_data = df[df['Production_Type'].isin(['Hydro_Water_Reservoir', 'Hydro_Pumped_Storage'])]
    output_file = os.path.join(output_folder, f"{data_year}_Total_Hydro_Installed_Capacity_per_Production_Type.csv")
    hydro_data.to_csv(output_file, index=False)
    print(f"Extracted hydro data from {input_file} to {output_file}")

extract_hydro_data(dispaSET_codes, availability_factors_folder_path, total_hydro_installed_capacity_folder_path, data_year)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
10. Availability Factors Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Availability Factor Folder is restored to its defoult state.
</div>

In [19]:
if os.path.exists(availability_factors_folder_path):
    shutil.rmtree(availability_factors_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, availability_factors_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors_backup/


In [20]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors_backup/ deleted successfully.
